# Chapter 6: Volumes: Attaching disk storage to containers

## Kubectl Commands

## Notes
### Introducing Volumes
* Kubernetes Volumes: Persistent Storage that can be shared in a pod
    * These are not top-level resources like pods
    * Volumes are accessible to all containers
    * Volumes are maintained even after a container restarts
    * For each container, you can mount the volume in any location of its filesystem
* Volume Types:
    * emptyDir: Single empty directory used for storing transient data
        * Best used for sharing files between containers running in the same pod
        * Can also be used to just store files too big for the containers memory
    * gitRepo: Volume initialized by checking out the contents of a Git repo
        * Once you make this type of volume then it won't be kept up to sync unless it's remade
        * Sidecar Container: Container that augments the operations of the main container of the pod
        * Once you make this type of volume then it won't be kept up to sync unless it's remade
    * hostPath: Used for mounting directories from the worker node's filesystem into the pod    
        * Does not get taken down when the pod is torn down
    * gcePersistentDisk: Used for mounting cloud provider storage

### Using Volumes to Share Data Between Containers
```bash
# Example volume mounting
apiVersion: v1
kind: Pod
metadata:
  name: fortune
spec:
containers:
- image: luksa/fortune
  name: html-generator
  volumeMounts:
  - name: html
    mountPath: /var/htdocs
volumes:
- name: html
  emptyDir: {}
```

## Using Persistent Storage
* Above storage volumes aren't good for if you have pods being rescheduled to another node
* Needs to be stored on Network-Attached Storage (NAS)
* Use the p-storage associated with your cluster:
  * AWS: awsElasticBlockStorage
  * Azure: azureDisk
  * GCP: gcePersistentDisk

## Using PersistentVolumes (PV)
* PS still kinda requires that the developer have knowledge on the underlying infrastructure
  * See NFS storage which utilizes user-defined server specs
* Access Flow:
  * Cluster admin setsup up some type of network storage
  * Admin creates a PersistentVolume by posting a PV descriptor to Kubernetes
  * User creates a PersistentVolumeClaim
  * Kubernetes finds and binds a PV of adequate size
  * User creates a pod with a volume referencing the PVC
* PersistentVolumes are resources like pods/nodes/replicators/etc.
* PersistentVolumes also don't belong to any namespace
  * PersistentVolumeClaims do though
* PersistentVolumeClaims is seperate from creating a pod
  * You want the same PVC to be available when the pod is being rescheduled
* AccessMode Abbreviations:
  * RWO: ReadWriteOnce
  * ROX: ReadOnlyMany
  * RWX: ReadWriteMany
  * Note that these are for # of worker nodes, not pods
* Reclaiming PersistentVolumes: 
  * Delete: Deletes the underlying storage
  * Recycle: Deletes the content + makes the volume available to future claiming

```bash
# Example PersistentVolume
apiVersion: v1
kind: PersistentVolume
metadata:
  name: mongodb-pv
spec:
  capacity:
    storage: 1Gi # PersistentVolume size
accessModes:
- ReadWriteOnce
- ReadOnlyMany
persistentVolumeReclaimPolicy: Retain # vs. Erased/Deleted
gcePersistentDisk:
 pdName: mongodb
fsType: ext4
```

```bash
# Example PersistentVolumeClaim
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: mongodb-pvc
spec:
  resources:
    requests:
      storage: 1Gi
  accessModes:
  - ReadWriteOnce
  storageClassName: ""
  ```

```bash
# Example Pod using a PersistentVolumeClaim
apiVersion: v1
kind: Pod
metadata:
  name: mongodb
spec:
  containers:
  - image: mongo
    name: mongodb
    volumeMounts:
    - name: mongodb-data
      mountPath: /data/db
    ports:
    - containerPort: 27017
      protocol: TCP
  volumes:
  - name: mongodb-data
    persistentVolumeClaim:
      claimName: mongodb-pvc
```

## Using StorageClasses
* Provisioning of PersistentVolumes can also be done automatically! 
  * This is so that the cluster admin doesn't have to do it
* Rather than having an admin pre-provision many PVs, they just define 1/2/n StorageClasses
* The StorageClasses let the system create a new PV when a PVC is requested

```bash
# Example StorageClass
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: fast
provisioner: kubernetes.io/gce-pd
parameters:
  zone: europe-west1-b
```

```bash
# Example PVC w/ dynamic provisioning
apiVersion: v1
        kind: PersistentVolumeClaim
        metadata:
          name: mongodb-pvc
spec:
  storageClassName: fast
  resources:
    requests:
      storage: 100Mi
  accessModes:
    - ReadWriteOnce
```

## Summary
* Create a multi-container pod and have the pod’s containers operate on the same files by adding a volume to the pod and mounting it in each container
* Use the emptyDir volume to store temporary, non-persistent data
* Use the gitRepo volume to easily populate a directory with the contents of a Git
repository at pod startup
* Use the hostPath volume to access files from the host node
* Mount external storage in a volume to persist pod data across pod restarts
* Decouple the pod from the storage infrastructure by using PersistentVolumes
and PersistentVolumeClaims
* Have PersistentVolumes of the desired (or the default) storage class dynami-
cally provisioned for each PersistentVolumeClaim
* Prevent the dynamic provisioner from interfering when you want the Persistent-
VolumeClaim to be bound to a pre-provisioned PersistentVolume